In [1]:
import os
import sys
import cv2
import numpy as np
from pathlib import Path

# Configuración para notebooks - sin usar __file__
# Asume que estamos en el mismo directorio que main.py
BASE_DIR = os.getcwd()
sys.path.append(BASE_DIR)
print(f"Usando directorio base: {BASE_DIR}")

# Intenta importar los módulos necesarios
try:
    from models.vertex_detector import VertexDetector
    from utils.contorno import obtener_contorno_imagen
    print("Módulos importados correctamente")
except ImportError as e:
    print(f"Error importando módulos: {e}")
    print("Verifica que estás en el directorio correcto y que los módulos existen")
    # Salir o manejar el error

def determinar_orientacion_etiqueta(etiqueta_detection, image, corners):
    """
    Determina la orientación de la imagen basada en la ubicación y
    dimensiones de la etiqueta detectada.
    """
    # Extraer coordenadas de la caja contenedora de la etiqueta
    x1, y1, x2, y2 = etiqueta_detection['bbox']
    
    # Calcular el centro de la etiqueta
    centro_x = (x1 + x2) // 2
    centro_y = (y1 + y2) // 2
    
    # Calcular dimensiones de la etiqueta
    ancho_etiqueta = x2 - x1
    alto_etiqueta = y2 - y1
    
    # Determinar si la etiqueta tiene orientación horizontal (ancho > alto)
    es_horizontal = ancho_etiqueta > alto_etiqueta
    
    # Determinar el lado donde está la etiqueta respecto al centro de la palanquilla
    if corners is not None and len(corners) == 4:
        # Calcular centro de la palanquilla
        centro_palanquilla_x = sum(corner[0] for corner in corners) / 4
        centro_palanquilla_y = sum(corner[1] for corner in corners) / 4
        
        # Calcular distancias relativas
        dx = centro_x - centro_palanquilla_x
        dy = centro_y - centro_palanquilla_y
        
        # Determinar el lado predominante
        if abs(dx) > abs(dy):
            # La etiqueta está a la izquierda o derecha
            if dx < 0:
                lado = 'izquierda'
                # CORRECCIÓN: Siempre rotar 90° si está a la izquierda
                angulo = 90
            else:
                lado = 'derecha'
                # Si está a la derecha, rotar -90° si horizontal, 180° si vertical
                angulo = -90 if es_horizontal else 180
        else:
            # La etiqueta está arriba o abajo
            if dy < 0:
                lado = 'arriba'
                # Si está arriba, rotar 180° si horizontal, 90° si vertical
                angulo = 180 if es_horizontal else 90
            else:
                lado = 'abajo'
                # Si está abajo, no rotar si horizontal, -90° si vertical
                angulo = 0 if es_horizontal else -90
    else:
        # Si no hay vertices, usar solo las dimensiones de la etiqueta
        lado = 'desconocido'
        if es_horizontal:
            angulo = 0  # Asumir que ya está en la orientación correcta
        else:
            angulo = 90  # Rotar para que el lado más largo sea horizontal
    
    print(f"Etiqueta detectada en lado: {lado}, orientación: {'horizontal' if es_horizontal else 'vertical'}")
    print(f"Ángulo de rotación calculado: {angulo}°")
    
    return angulo, lado

# Función para procesar una imagen individual
def procesar_imagen(imagen_path, output_dir="pruebas_etiquetas"):
    """
    Procesa una sola imagen para probar la detección y rotación de etiquetas
    """
    # Crear directorio de salida
    os.makedirs(output_dir, exist_ok=True)
    
    # Configurar modelo - AJUSTA ESTA RUTA según donde tengas tu modelo
    vertex_model_path = r"D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.pt"
    print(f"Cargando modelo desde: {vertex_model_path}")
    
    # Intentar cargar el modelo
    try:
        vertex_detector = VertexDetector(vertex_model_path)
    except Exception as e:
        print(f"Error cargando el modelo: {e}")
        return
    
    # Cargar imagen
    image = cv2.imread(imagen_path)
    if image is None:
        print(f"Error: No se pudo cargar la imagen {imagen_path}")
        return
    
    # Nombre base de la imagen
    nombre_base = os.path.basename(imagen_path)
    nombre, ext = os.path.splitext(nombre_base)
    
    # Hacer una detección inicial de vértices
    try:
        print("\nDetectando contorno inicial de palanquilla...")
        initial_vertices, contorno_principal, _ = obtener_contorno_imagen(
            image, vertex_detector.model, vertex_detector.conf_threshold, 
            target_class=vertex_detector.target_class)
        print(f"Contorno detectado con {len(initial_vertices) if initial_vertices is not None else 0} vértices")
    except Exception as e:
        print(f"Error en detección inicial de vértices: {e}")
        initial_vertices = None
    
    # Detectar etiquetas
    print("\nBuscando etiquetas...")
    etiqueta_detections = []
    try:
        # Intentar un umbral de confianza más bajo para detectar mejor las etiquetas
        conf_threshold_etiqueta = 0.1  # Más bajo que el predeterminado
        vertex_result = vertex_detector.model.predict(
            image, conf=conf_threshold_etiqueta, device=vertex_detector.device)[0]
        
        if hasattr(vertex_detector.model, "names") and vertex_result.boxes is not None:
            class_names = vertex_detector.model.names
            print(f"Clases disponibles en el modelo: {class_names}")
            
            etiqueta_class_id = None
            # Buscar la clase "etiqueta"
            for id, name in class_names.items():
                if isinstance(name, str) and name.lower() == "etiqueta":
                    etiqueta_class_id = id
                    print(f"ID de clase para 'etiqueta' encontrado: {etiqueta_class_id}")
                    break
            
            # Si no encontramos "etiqueta", probar con class_0
            if etiqueta_class_id is None and 0 in class_names:
                etiqueta_class_id = 0
                print(f"No se encontró 'etiqueta' explícitamente. Usando class_0 como etiqueta, nombre: {class_names[0]}")
            
            if etiqueta_class_id is not None:
                boxes = vertex_result.boxes
                print(f"Total de detecciones: {len(boxes)}")
                
                for i, box in enumerate(boxes):
                    cls_id = int(box.cls[0].item())
                    conf = float(box.conf[0].item())
                    print(f"Detección #{i+1}: Clase {cls_id}, Confianza {conf:.2f}")
                    
                    if cls_id == etiqueta_class_id:
                        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                        print(f"ETIQUETA detectada con confianza {conf:.2f} en bbox: ({x1}, {y1}, {x2}, {y2})")
                        
                        etiqueta_detections.append({
                            'bbox': (x1, y1, x2, y2),
                            'conf': conf,
                            'class': 'etiqueta',
                            'cls_id': cls_id
                        })
                
                print(f"Total etiquetas encontradas: {len(etiqueta_detections)}")
    except Exception as e:
        print(f"Error al buscar etiquetas: {e}")
        import traceback
        traceback.print_exc()
    
    # Crear imagen de visualización
    vis_img = image.copy()
    
    # Dibujar los vértices detectados
    if initial_vertices is not None:
        cv2.polylines(vis_img, [np.array(initial_vertices)], True, (0, 255, 0), 2)
        
    # Dibujar las etiquetas detectadas
    for i, etiqueta in enumerate(etiqueta_detections):
        x1, y1, x2, y2 = etiqueta['bbox']
        cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(vis_img, f"Etiqueta {i+1}", (x1, y1-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Guardar imagen con detecciones
    detections_path = os.path.join(output_dir, f"{nombre}_detecciones{ext}")
    cv2.imwrite(detections_path, vis_img)
    print(f"Imagen con detecciones guardada en: {detections_path}")
    
    # Si se encontraron etiquetas, procesar rotación
    if etiqueta_detections:
        # Ordenar etiquetas por confianza
        etiqueta_detections.sort(key=lambda x: x['conf'], reverse=True)
        mejor_etiqueta = etiqueta_detections[0]
        
        # Determinar ángulo de rotación
        angulo_rotacion, lado_etiqueta = determinar_orientacion_etiqueta(
            mejor_etiqueta, image, initial_vertices)
        
        # Visualizar información de la etiqueta
        etiqueta_img = image.copy()
        x1, y1, x2, y2 = mejor_etiqueta['bbox']
        
        # Dibujar la etiqueta principal
        cv2.rectangle(etiqueta_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Dibujar información sobre la etiqueta
        ancho = x2 - x1
        alto = y2 - y1
        es_horizontal = ancho > alto
        
        info_text = [
            f"Lado: {lado_etiqueta}",
            f"Orientacion: {'Horizontal' if es_horizontal else 'Vertical'}",
            f"Ancho x Alto: {ancho} x {alto}",
            f"Rotacion: {angulo_rotacion} grados"
        ]
        
        for i, text in enumerate(info_text):
            cv2.putText(etiqueta_img, text, (10, 30 + 30*i), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Guardar imagen con info de etiqueta
        etiqueta_path = os.path.join(output_dir, f"{nombre}_info_etiqueta{ext}")
        cv2.imwrite(etiqueta_path, etiqueta_img)
        print(f"Imagen con info de etiqueta guardada en: {etiqueta_path}")
        
        # Aplicar rotación
        if angulo_rotacion != 0:
            h, w = image.shape[:2]
            center = (w // 2, h // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angulo_rotacion, 1.0)
            rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_CUBIC)
            
            # Guardar imagen rotada
            rotated_path = os.path.join(output_dir, f"{nombre}_rotada{ext}")
            cv2.imwrite(rotated_path, rotated_image)
            print(f"Imagen rotada guardada en: {rotated_path}")
            
            # Crear imagen comparativa lado a lado
            h_comp = min(h, 800)
            w_comp = int(w * (h_comp / h))
            
            original_resized = cv2.resize(image, (w_comp, h_comp))
            rotada_resized = cv2.resize(rotated_image, (w_comp, h_comp))
            
            # Comparación lado a lado
            comparison = np.zeros((h_comp, w_comp*2, 3), dtype=np.uint8)
            comparison[:, :w_comp] = original_resized
            comparison[:, w_comp:] = rotada_resized
            
            # Añadir etiquetas
            cv2.putText(comparison, "ORIGINAL", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(comparison, f"ROTADA {angulo_rotacion}°", (w_comp+10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Guardar comparación
            comp_path = os.path.join(output_dir, f"{nombre}_comparacion{ext}")
            cv2.imwrite(comp_path, comparison)
            print(f"Imagen comparativa guardada en: {comp_path}")
    else:
        print("No se detectaron etiquetas en la imagen.")

# Función para procesar un directorio
def procesar_directorio(directorio_path, output_dir=None):
    """
    Procesa todas las imágenes en un directorio
    """
    if output_dir is None:
        output_dir = os.path.join(os.getcwd(), "resultados_etiquetas")
    
    os.makedirs(output_dir, exist_ok=True)
    print(f"Directorio de salida: {output_dir}")
    
    # Extensiones válidas
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff']
    
    # Encontrar imágenes
    image_files = []
    for ext in valid_extensions:
        image_files.extend(list(Path(directorio_path).glob(f'*{ext}')))
        image_files.extend(list(Path(directorio_path).glob(f'*{ext.upper()}')))
    
    print(f"Encontradas {len(image_files)} imágenes en {directorio_path}")
    
    # Procesar cada imagen
    for i, img_path in enumerate(image_files):
        print(f"\n[{i+1}/{len(image_files)}] Procesando: {img_path.name}")
        
        # Crear carpeta específica para esta imagen
        img_output_dir = os.path.join(output_dir, img_path.stem)
        os.makedirs(img_output_dir, exist_ok=True)
        
        # Procesar
        procesar_imagen(str(img_path), img_output_dir)
    
    print("\nProcesamiento completado.")

directorio_path = r"D:\Trabajo modelos\PACC\YOLOv12 - copia\pruebas diagonales"
procesar_directorio(directorio_path)

Usando directorio base: d:\Trabajo modelos\PACC\YOLOv12 - copia\modulado
WARNING Error decoding JSON from C:\Users\locop\AppData\Roaming\Ultralytics\persistent_cache.json. Starting with an empty dictionary.
Módulos importados correctamente
Directorio de salida: d:\Trabajo modelos\PACC\YOLOv12 - copia\modulado\resultados_etiquetas
Encontradas 6 imágenes en D:\Trabajo modelos\PACC\YOLOv12 - copia\pruebas diagonales

[1/6] Procesando: 1era_imagen_png.rf.bd7bc392b4bb48111f1fbac79c898070.jpg
Cargando modelo desde: D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.pt
Buscando archivo YAML en: D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.yaml
Clases cargadas desde D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.yaml: ['etiqueta', 'palanquilla']
Clase objetivo 'palanquilla' encontrada en índice 1
Usando dispositivo: cuda:0
Modelo YOLO para detección de vértices cargado desde: D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.pt
Clase o